<a href="https://colab.research.google.com/github/peterbabulik/QuantumWalker/blob/main/BellState.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit qiskit-ibm-runtime qiskit-aer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.9/359.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mpl_colors
import time
import os
import traceback

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

# Modern IBM Quantum access
try:
    from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Session
    from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
    from qiskit_ibm_runtime.ibm_backend import IBMBackend as IBMRuntimeBackend
    qiskit_runtime_available = True
    print("Successfully imported Qiskit Runtime modules.")
except ImportError:
    print("CRITICAL WARNING: qiskit_ibm_runtime could not be imported.")
    QiskitRuntimeService = None; Sampler = None; Session = None; generate_preset_pass_manager = None
    IBMRuntimeBackend = None
    qiskit_runtime_available = False

# --- Parameters ---
NUM_QUBITS_PROBE_TEST = 2 # For Bell state + H probe

# --- IBM Quantum API Token ---
IBM_QUANTUM_TOKEN_DIRECT = 'API GO HERE'
IBM_QUANTUM_CHANNEL_DIRECT = 'ibm_quantum'
IBM_QUANTUM_INSTANCE_DIRECT = 'ibm-q/open/main'

service = None
if qiskit_runtime_available:
    try:
        print(f"Initializing IBM QiskitRuntimeService with token...")
        service = QiskitRuntimeService(channel=IBM_QUANTUM_CHANNEL_DIRECT, instance=IBM_QUANTUM_INSTANCE_DIRECT, token=IBM_QUANTUM_TOKEN_DIRECT)
        print("IBM QiskitRuntimeService initialized.")
    except Exception as e: print(f"ERROR initializing QiskitRuntimeService: {e}"); service = None
else: print("qiskit_ibm_runtime not available.")

# --- Select Backend ---
backend = None
backend_name_to_print = "AerSimulator (local)"
backend_target_name = 'ibm_brisbane'

if service:
    try:
        print(f"\nAttempting to get QPU backend: {backend_target_name}...")
        qpu_obj = service.backend(backend_target_name)
        if qpu_obj and qpu_obj.status().operational:
            backend = qpu_obj; backend_name_to_print = backend.name
            print(f"Selected QPU: {backend.name} (Status: {backend.status().status_msg})")
        else: print(f"QPU {backend_target_name} not found/operational."); backend = None
    except Exception as e: print(f"Could not get QPU '{backend_target_name}': {e}"); backend = None

if backend is None:
    print("\nNo QPU. Falling back to AerSimulator.")
    backend = AerSimulator(); backend_name_to_print = "AerSimulator (local)"

# --- Define Bell State + H Probe Test Circuit ---
print(f"\nBuilding Bell State + H Probe test circuit ({NUM_QUBITS_PROBE_TEST} qubits)...")
qr = QuantumRegister(NUM_QUBITS_PROBE_TEST, 'q')
cr = ClassicalRegister(NUM_QUBITS_PROBE_TEST, 'c_out')
qc_probe_test = QuantumCircuit(qr, cr, name="Bell_H_Probe_Test")

# 1. Create Bell State |Φ+> = (|00> + |11>)/sqrt(2)
qc_probe_test.h(qr[0])
qc_probe_test.cx(qr[0], qr[1])
qc_probe_test.barrier(label="bell_state")

# 2. Probe with H on one qubit of the pair
qc_probe_test.h(qr[0]) # Apply H to the first qubit
qc_probe_test.barrier(label="H_probe")

# 3. Measure
qc_probe_test.measure(qr[0], cr[0])
qc_probe_test.measure(qr[1], cr[1])

print("\n--- Generated Bell State + H Probe Test Circuit ---")
try: print(qc_probe_test.draw(output='text', fold=-1))
except Exception as e_draw: print(f"Could not draw circuit: {e_draw}")


# --- Transpile and Run ---
print(f"\n--- Transpiling and Running on {backend_name_to_print} ---")
counts = {}
try:
    shots = 4096
    isa_circuit = qc_probe_test
    is_runtime_target = (qiskit_runtime_available and service and backend and
                         IBMRuntimeBackend and isinstance(backend, IBMRuntimeBackend))

    if is_runtime_target and generate_preset_pass_manager:
        print("Transpiling for Qiskit Runtime backend...")
        pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
        isa_circuit = pm.run(qc_probe_test)
        print(f"Circuit depth original: {qc_probe_test.depth()}, transpiled: {isa_circuit.depth()}")
    elif isinstance(backend, AerSimulator):
        print(f"Transpiling for {backend_name_to_print}...")
        isa_circuit = transpile(qc_probe_test, backend=backend, optimization_level=1)
        print(f"Circuit depth original: {qc_probe_test.depth()}, transpiled: {isa_circuit.depth()}")
    else:
        print(f"Using original circuit (depth: {qc_probe_test.depth()}).")

    if is_runtime_target and Sampler and Session:
        print(f"Using SamplerV2 for {backend.name}")
        with Session(backend=backend) as session:
            sampler = Sampler()
            job = sampler.run([(isa_circuit,)], shots=shots)
            print(f"Job ID: {job.job_id()} submitted.")
            result = job.result()
            print("Processing SamplerV2 results...")
            if result and len(result) > 0:
                pub_result = result[0]
                target_creg_name = cr.name
                if hasattr(pub_result.data, target_creg_name):
                    bit_array_obj = getattr(pub_result.data, target_creg_name)
                    if bit_array_obj: counts = bit_array_obj.get_counts()
                    else: print(f"WARNING: BitArray for '{target_creg_name}' is None."); counts = {}
                elif hasattr(pub_result.data, 'meas'):
                    bit_array_obj = pub_result.data.meas
                    if bit_array_obj: counts = bit_array_obj.get_counts()
                    else: print("WARNING: BitArray for 'meas' is None."); counts = {}
                else: print(f"WARNING: No data field '{target_creg_name}' or 'meas'. Attrs: {dir(pub_result.data)}"); counts = {}
            else: print("ERROR: Job result is None/empty."); counts = {}
    elif isinstance(backend, AerSimulator):
        print(f"Using legacy backend.run() for {backend_name_to_print}")
        job = backend.run(isa_circuit, shots=shots)
        counts = job.result().get_counts(0)
    else: print("ERROR: No suitable execution path."); counts = {}

    print("\n--- Results ---"); print("Counts:", counts)
    if counts:
        plot_histogram(counts, title=f"Bell State + H Probe on {backend_name_to_print} (Qubits={NUM_QUBITS_PROBE_TEST})")
        plt.show()
    else: print("No counts data to plot.")

except Exception as e: print(f"Error: {e}"); traceback.print_exc()
finally: print(f"\nEXPERIMENT CONCLUDED. Invalidate token '{IBM_QUANTUM_TOKEN_DIRECT[:10]}...' now.")

Successfully imported Qiskit Runtime modules.
Initializing IBM QiskitRuntimeService with token...


<ipython-input-2-92746616aa51>:38: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel=IBM_QUANTUM_CHANNEL_DIRECT, instance=IBM_QUANTUM_INSTANCE_DIRECT, token=IBM_QUANTUM_TOKEN_DIRECT)


IBM QiskitRuntimeService initialized.

Attempting to get QPU backend: ibm_brisbane...
Selected QPU: ibm_brisbane (Status: active)

Building Bell State + H Probe test circuit (2 qubits)...

--- Generated Bell State + H Probe Test Circuit ---
         ┌───┐      bell_state ┌───┐ H_probe ┌─┐   
    q_0: ┤ H ├──■───────░──────┤ H ├────░────┤M├───
         └───┘┌─┴─┐     ░      └───┘    ░    └╥┘┌─┐
    q_1: ─────┤ X ├─────░───────────────░─────╫─┤M├
              └───┘     ░               ░     ║ └╥┘
c_out: 2/═════════════════════════════════════╩══╩═
                                              0  1 

--- Transpiling and Running on ibm_brisbane ---
Transpiling for Qiskit Runtime backend...
Circuit depth original: 4, transpiled: 11
Using SamplerV2 for ibm_brisbane
Job ID: d0gztrbfbx30008w5xfg submitted.
Processing SamplerV2 results...

--- Results ---
Counts: {'11': 1126, '01': 1087, '10': 880, '00': 1003}

EXPERIMENT CONCLUDED. Invalidate token '99dd752798...' now.
